In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
import os
code_path = "/content/gdrive/MyDrive/CSE512-HW2/"
os.chdir(code_path)

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy

import random


In [ ]:
corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()
corpus = ' '.join(corpus)

def clean_word(word):
    word = word.lower()
    for punctuation in ['"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        word = word.split(punctuation)[0]
    return word



corpus = [clean_word(word) for word in corpus.split()]
corpus = [word for word in corpus if len(word) > 0]
print(corpus[:25])
D = len(corpus)
print('corpus len: ',D)

['alice', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', '3', 'contents', 'chapter', 'i', 'down', 'the', 'rabbit', 'chapter', 'ii', 'the', 'pool', 'of', 'tears', 'chapter']
corpus len:  25320


In [ ]:
tokenize = {}
dictionary = []
token = 0
for word in corpus:
    if word not in tokenize.keys():
        tokenize[word] = token
        dictionary.append(word)
        token += 1
    
V = len(dictionary)
print('dictionary size (number of distinct words): ', V)



dictionary size (number of distinct words):  2501


In [ ]:
#past word as feature

posterior_1word = np.zeros((V, V))
prior = np.zeros(V)
# Calculate the prior of the words.
for i in range(len(corpus)):
    prior[tokenize[corpus[i]]] += 1
    if i > 0:
       posterior_1word[tokenize[corpus[i-1]]][tokenize[corpus[i]]] += 1

posterior_1word = posterior_1word / prior
prior = prior / len(corpus)

def get_likelihood_2gram(word):
    likelihood = posterior_1word[tokenize[word], :] * prior
    return(likelihood)
def pred_2gram(word):
    likelihood = get_likelihood_2gram(word)
    i = np.argmax(likelihood)
    return(dictionary[i], likelihood[i])
print(pred_2gram('alice'))
print(pred_2gram('the'))
print(pred_2gram('cheshire'))
print(pred_2gram('mock'))
    

('was', 0.0007109004739336493)
('queen', 0.0027646129541864135)
('cat', 0.00019747235387045816)
('turtle', 0.0022511848341232226)


In [ ]:
# Using the likelihoods computed from the bigram classiffer, and starting with a seed word "alice", 
# generate the next 25 words by always picking the most likely next word.
word = "alice"
article = word
for i in range(25):
    word, _ = pred_2gram(word)
    article = article + " " + word
print(article)


alice was a little thing i can remember ever saw in a little thing i can remember ever saw in a little thing i can remember


In [ ]:
# Using random choices method
word = "alice"
article = word
for i in range(25):
    likelihood = get_likelihood_2gram(word)
    word = random.choices(dictionary, weights = likelihood)[0]
    article = article + " " + word
print(article)

alice and large again your majesty the gryphon and then thought must have a minute or drink something or not to be herself very gravely think


In [ ]:
# Calculate the acc of 1 word.
positive = 0
for i in range(len(corpus) - 1):
    if pred_2gram(corpus[i])[0] == corpus[i+1]:
        positive += 1
print("The accuracy:", positive / (len(corpus)-1))

The accuracy: 0.2453493423910897


In [ ]:
#past 2 words as features

posterior_2words = np.zeros((V, V))
for i in range(0, len(corpus) - 2):
    posterior_2words[tokenize[corpus[i]]][tokenize[corpus[i + 2]]] += 1 
posterior_2words /= prior

posterior_2gram = np.vstack([posterior_1word,posterior_2words])



def get_likelihood_3gram(word2ago,word1ago):
    likelihood = posterior_1word[tokenize[word1ago], :] * posterior_2words[tokenize[word2ago], :] * prior
    return likelihood
def pred_3gram(word2ago,word1ago):
    likelihood = get_likelihood_3gram(word2ago,word1ago)
    i = np.argmax(likelihood)
    return dictionary[i], likelihood[i]
print(pred_3gram('pack','of'))
print(pred_3gram('the','mad'))
print(pred_3gram('she','jumped'))

    

('cards', 3.0)
('you', 0.14447592067988668)
('up', 0.5416666666666666)


In [ ]:
first_word = "alice"
second_word = "was"
article = first_word + " " + second_word
for i in range(25):
    new, _ = pred_3gram(first_word, second_word)
    article = article + " " + new
    first_word = second_word
    second_word = new
print(article)


alice was not easy to take this young lady tells us a story afraid i am i ah that the queen who was peeping anxiously into its


In [ ]:
# Using random choices method
first_word = "alice"
second_word = "was"
article = first_word + " " + second_word
for i in range(25):
    likelihood = get_likelihood_3gram(first_word, second_word)
    new = random.choices(dictionary, weights = likelihood)[0]
    article = article + " " + new
    first_word = second_word
    second_word = new
print(article)

alice was perfectly round she swam lazily about in a moment she felt certain it must know the song perhaps heard something splashing about in some of


In [ ]:
# Calculate the acc of 2 word.
positive = 0
for i in range(len(corpus) - 2):
    if pred_3gram(corpus[i], corpus[i + 1])[0] == corpus[i+2]:
        positive += 1
print("The accuracy:", positive / (len(corpus)-2))

The accuracy: 0.5047397108776365
